# Data Warehouse Truven QA - Claim Header

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

In [3]:
year_df = pd.read_sql('select distinct year from dw_staging.claim_header_1_prt_truv;', con=connection)
tables = ['ccaes', 'mdcrs', 'mdcro', 'ccaeo']

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_header table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the truven schema. 


In [3]:
query = ''' drop table if exists qa_reporting.dw_truv_claim_header_counts;
create table qa_reporting.dw_truv_claim_header_counts
(
    calendar_year int,
    table_src text,
    dw_row_count int,
    src_row_count int,
    row_count_diff int,
    row_count_diff_percentage float,
    dw_uth_clm_id_count int,
    dw_src_clm_id_count int,
    src_clm_count int,
    clm_count_diff int,
    clm_count_percentage float,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [7]:
with connection.cursor() as cursor:
    for year in tqdm(year_df['year']):
        query = f'''
        insert into qa_reporting.dw_truv_claim_header_counts
        (calendar_year, table_src, dw_row_count, date_generated)
        select year, table_id_src, count(*), current_date
        from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1, 2
        '''
        
        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_clm_id_count = count
        from (
            select year, table_id_src,  count(distinct uth_claim_id) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1, 2 ) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_clm_id_count = count
        from (
            select year, table_id_src,  count(distinct claim_id_src) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1, 2 ) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_mbr_id_count = count
        from (
            select year, table_id_src,  count(distinct uth_member_id) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1, 2 ) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)
        
        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_mbr_id_count = count
        from (
            select year, table_id_src,   count(distinct member_id_src) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1, 2 ) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        insert into qa_reporting.dw_truv_claim_header_counts
        (calendar_year, table_src, dw_row_count, date_generated)
        select year, 'ALL' table_id_src, count(*), current_date
        from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1
        '''
        
        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_clm_id_count = count
        from (
            select year, 'ALL' table_id_src,  count(distinct uth_claim_id) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1 ) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_clm_id_count = count
        from (
            select year, 'ALL' table_id_src,  count(distinct claim_id_src) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_mbr_id_count = count
        from (
            select year, 'ALL' table_id_src,  count(distinct uth_member_id) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1 ) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)
        
        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_mbr_id_count = count
        from (
            select year, 'ALL' table_id_src,   count(distinct member_id_src) as count 
            from dw_staging.claim_header_1_prt_truv
        where year = {year}
        group by 1 ) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
100%|██████████| 12/12 [09:58<00:00, 49.88s/it]


In [9]:
with connection.cursor() as cursor:
    query = '''update qa_reporting.dw_truv_claim_header_counts a
    set src_row_count = b.row_count,
    row_count_diff = dw_row_count - b.row_count,
    row_count_diff_percentage = 100. * abs( dw_row_count - b.row_count) / b.row_count,
    src_clm_count = clm_count,
    clm_count_diff = dw_uth_clm_id_count - b.clm_count,
    clm_count_percentage = 100. * abs(dw_uth_clm_id_count - b.clm_count) / b.clm_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = dw_uth_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(dw_uth_mbr_id_count - b.pat_count) / b.pat_count
    from qa_reporting.truven_counts b
    where a.calendar_year = b.year
    and a.table_src = b.table_name
    ;
    '''

    cursor.execute(query)

ccaeo counts need to be run not using Python due to how long queries using this table run overall. Psycopg or Greenplum has a time limit for queries that are executed in this manner that is about 2 hours. The query may continue running on database server even if the connection to the server is closed. This means if this query is executed from here, you can check the status of the query using Greenplum (through a seperate query).

```
with truven_claims as (
            select year, 'ccaes' as table_name, enrolid, claim_id_derv
            from staging_clean.ccaes_etl
            union
            select year, 'ccaeo' as table_name, enrolid, claim_id_derv
            from staging_clean.ccaeo_etl
            union
            select year, 'mdcrs' as table_name, enrolid, claim_id_derv
            from staging_clean.mdcrs_etl
            union
            select year, 'mdcro' as table_name, enrolid, claim_id_derv
            from staging_clean.mdcro_etl
        ),
        truven_claims_counts as (
            select year, 'ALL' table_name, count(*) row_count, count(distinct enrolid) mbr_count, count(distinct claim_id_derv) clm_count
            from truven_claims
            group by 1,2
        )
        update qa_reporting.dw_truv_claim_header_counts a
        set src_row_count = b.row_count,
        row_count_diff = dw_row_count - b.row_count,
        row_count_diff_percentage = 100. * abs( dw_row_count - b.row_count) / b.row_count,
        src_clm_count = clm_count,
        clm_count_diff = dw_uth_clm_id_count - b.clm_count,
        clm_count_percentage = 100. * abs(dw_uth_clm_id_count - b.clm_count) / b.clm_count,
        src_mbr_count = b.mbr_count,
        mbr_count_diff = dw_uth_mbr_id_count - b.mbr_count,
        mbr_count_percentage = 100. * abs(dw_uth_mbr_id_count - b.mbr_count) / b.mbr_count
        from truven_claims_counts b
        where a.calendar_year = b.year
        and a.table_src = b.table_name
        ;
```

Note that the source row count from the o and s tables from truven are at detail level. This means that the row count from the individual tables are going to larger than the row count in the claim header table. If we use the query above, it will take the count of unique rows (based on year, table name, claim id, member id) which may be a more accurate method to evaluate the row count for the claim header table.

In [3]:
df = pd.read_sql('select * from qa_reporting.dw_truv_claim_header_counts;', con=connection)
df.head()

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,2018,ALL,290723506,290789941,-66435,0.022846,290425796,290412151,290492110,-66314,0.022828,22547314,22547314,22547774,-460,0.002040,2023-04-21
1,2018,ccaes,10980355,38984126,-28003771,71.833779,10980355,10975904,10976708,3647,0.033225,1029737,1029737,1029737,0,0.000000,2023-04-21
2,2018,mdcrs,2343698,6992226,-4648528,66.481375,2343698,2341981,2342184,1514,0.064641,147674,147674,147674,0,0.000000,2023-04-21
3,2018,mdcro,26096056,59193949,-33097893,55.914318,26096056,26092957,26105522,-9466,0.036261,1057407,1057407,1057428,-21,0.001986,2023-04-21
4,2018,ccaeo,251303397,589022269,-337718872,57.335502,251303397,251299019,251365523,-62126,0.024715,21528585,21528585,21529034,-449,0.002086,2023-04-21


In [4]:
df[df['table_src'] == 'ALL']

,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,2018,ALL,290723506,290789941,-66435,0.022846,290425796,290412151,290492110,-66314,0.022828,22547314,22547314,22547774,-460,0.002040,2023-04-21
5,2020,ALL,271605426,271653875,-48449,0.017835,271405951,271395927,271454354,-48403,0.017831,20238560,20238560,20239042,-482,0.002382,2023-04-21
10,2021,ALL,288698899,288791345,-92446,0.032011,288506737,288497809,288599077,-92340,0.031996,20042311,20042311,20044196,-1885,0.009404,2023-04-21
15,2014,ALL,505258663,505315935,-57272,0.011334,504826402,504810286,504883546,-57144,0.011318,39662807,39662807,39663323,-516,0.001301,2023-04-21
20,2019,ALL,299644157,299676211,-32054,0.010696,299394734,299383819,299426746,-32012,0.010691,21777984,21777984,21778393,-409,0.001878,2023-04-21
25,2011,ALL,605491625,605479143,12482,0.002062,604833166,604820682,604820682,12484,0.002064,47089265,47089265,47089265,0,0.000000,2023-04-21
26,2012,ALL,607339141,607373049,-33908,0.005583,606721289,606707165,606755083,-33794,0.005570,47494758,47494758,47495022,-264,0.000556,2023-04-21
35,2022,ALL,132274111,132327243,-53132,0.040152,132181344,132181175,132234183,-52839,0.039959,15144527,15144527,15154145,-9618,0.063468,2023-04-21
40,2016,ALL,329529635,329515942,13693,0.004155,329200320,329186607,329186623,13697,0.004161,24725600,24725600,24725600,0,0.000000,2023-04-21
45,2017,ALL,293553201,293589258,-36057,0.012281,293290364,293275774,293326352,-35988,0.012269,22248809,22248809,22249093,-284,0.001276,2023-04-21


In [18]:
df['row_count_diff_percentage'].describe()

count    60.000000
mean     50.839985
std      26.312083
min       0.002062
25%      55.208591
50%      58.161286
75%      68.909598
max      73.379011
Name: row_count_diff_percentage, dtype: float64

In [19]:
df['clm_count_percentage'].describe()

count    60.000000
mean      0.028526
std       0.070955
min       0.000987
25%       0.009099
50%       0.018172
75%       0.030717
max       0.559619
Name: clm_count_percentage, dtype: float64

In [20]:
df['mbr_count_percentage'].describe()

count    60.000000
mean      0.025437
std       0.161949
min       0.000000
25%       0.000000
50%       0.000813
75%       0.002257
max       1.254483
Name: mbr_count_percentage, dtype: float64

## Claim Type

In [26]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_claim_type_count;
    with truv_claims as (
        select year, claim_id_derv, facprof
        from truven.ccaeo
        union
        select year, claim_id_derv, facprof
        from truven.ccaes
        union
        select year, claim_id_derv, facprof
        from truven.mdcro
        union
        select year, claim_id_derv, facprof
        from truven.mdcrs
    )
select year, facprof, count(*)
into qa_reporting.truv_claim_type_count
from truv_claims
group by 1,2
    '''
    cursor.execute(query)

In [23]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_claim_type_count;
select year, claim_type, count(*)
into qa_reporting.dw_truv_claim_type_count
from dw_staging.claim_header
group by 1,2
    '''
    cursor.execute(query)

In [34]:
query = f'''
select a.year, a.claim_type, a.count as dw_claim_type_count, b.count as src_claim_type_count, 
        a.count-b.count as claim_type_count_difference, 
         100. * abs(a.count-b.count) / b.count as claim_type_count_difference_percentage
from qa_reporting.dw_truv_claim_type_count a
full outer join qa_reporting.truv_claim_type_count b
on a.year =b.year
and a.claim_type = b.facprof
'''

pd.read_sql(query, con=connection).sort_values(['year', 'claim_type'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,claim_type,dw_claim_type_count,src_claim_type_count,claim_type_count_difference,claim_type_count_difference_percentage
0,2011,F,74242621,74022443,220178,0.297448
8,2011,P,531249004,530798241,450763,0.084922
1,2012,F,74154300,73946444,207856,0.281090
11,2012,P,533184841,532808641,376200,0.070607
10,2013,F,58878891,58740345,138546,0.235862
20,2013,P,429119051,428822352,296699,0.069189
16,2014,F,62955864,62858795,97069,0.154424
5,2014,P,442302799,442024753,278046,0.062903
2,2015,F,40273680,40208649,65031,0.161734
14,2015,P,280980268,280818543,161725,0.057591


## Costs

### Sum

In [ ]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_total;
    with truv_claims as (
        select year, claim_id_derv, netpay, pay
        from truven.ccaeo
        union
        select year, claim_id_derv, netpay, pay
        from truven.ccaes
        union
        select year, claim_id_derv, netpay, pay
        from truven.mdcro
        union
        select year, claim_id_derv, netpay, pay
        from truven.mdcrs
    )
select year, 'ALL' as table_name, 
        sum(netpay) total_netpay,
        sum(pay) total_pay
into qa_reporting.truv_costs_total
from truv_claims
group by 1
    '''
    cursor.execute(query)

In [10]:
for table in tables:
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_total
    select year, '{table}' as table_name, sum(netpay) total_netpay, sum(pay) total_pay
    from truven.{table}
    group by 1
        '''
        cursor.execute(query)

In [3]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_total;
select year, table_id_src,
        sum(total_charge_amount) as total_charge_amount,
        sum(total_allowed_amount) as total_allowed_amount, 
        sum(total_paid_amount) as total_paid_amount
into qa_reporting.dw_truv_costs_total
from dw_staging.claim_header
group by 1,2
    '''
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
    query = '''insert into qa_reporting.dw_truv_costs_total
select year, 'ALL',
        sum(total_charge_amount) as total_charge_amount,
        sum(total_allowed_amount) as total_allowed_amount, 
        sum(total_paid_amount) as total_paid_amount
from dw_staging.claim_header
group by 1
    '''
    cursor.execute(query)

In [17]:
query = f'''
select a.year, b.table_name, 
        a.total_paid_amount as dw_total_paid, b.total_netpay as src_total_paid, 
        a.total_paid_amount-b.total_netpay as total_paid_difference, 
        100. * abs(a.total_paid_amount-b.total_netpay) / b.total_netpay as total_paid_difference_percentage,
        a.total_allowed_amount as dw_total_allowed, b.total_pay as src_total_allowed,
        a.total_allowed_amount-b.total_pay as total_allowed_difference, 
        100. * abs(a.total_allowed_amount-b.total_pay) / b.total_pay as total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_total a
full outer join qa_reporting.truv_costs_total b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,table_name,dw_total_paid,src_total_paid,total_paid_difference,total_paid_difference_percentage,dw_total_allowed,src_total_allowed,total_allowed_difference,total_allowed_difference_percentage
32,2011,ALL,1.611636e+11,1.535983e+11,7.565246e+09,4.925344,2.267115e+11,2.152597e+11,1.145178e+10,5.319985
5,2011,ccaeo,9.791353e+10,9.793091e+10,-1.737835e+07,0.017746,1.204639e+11,1.204835e+11,-1.959208e+07,0.016261
22,2011,ccaes,4.848342e+10,4.853014e+10,-4.671581e+07,0.096261,5.304501e+10,5.309506e+10,-5.004118e+07,0.094248
15,2011,mdcro,9.648785e+09,9.647447e+09,1.338554e+06,0.013875,3.282233e+10,3.281620e+10,6.131274e+06,0.018684
16,2011,mdcrs,5.117819e+09,5.117239e+09,5.804408e+05,0.011343,2.038027e+10,2.037895e+10,1.317226e+06,0.006464
2,2012,ALL,1.656924e+11,1.577249e+11,7.967511e+09,5.051524,2.336460e+11,2.213655e+11,1.228048e+10,5.547606
44,2012,ccaeo,1.023849e+11,1.024156e+11,-3.068512e+07,0.029961,1.277449e+11,1.277824e+11,-3.744669e+07,0.029305
53,2012,ccaes,4.997265e+10,5.001377e+10,-4.112837e+07,0.082234,5.468143e+10,5.472552e+10,-4.408268e+07,0.080552
40,2012,mdcro,8.678780e+09,8.678058e+09,7.220791e+05,0.008321,3.214452e+10,3.214170e+10,2.813778e+06,0.008754
9,2012,mdcrs,4.656050e+09,4.655846e+09,2.041928e+05,0.004386,1.907508e+10,1.907359e+10,1.487018e+06,0.007796


### Min

In [ ]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_min;
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.ccaeo
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.ccaes
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.mdcro
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.mdcrs
        group by 1,2
    )
select year, 'ALL' as table_name, 
        min(netpay) min_netpay,
        min(pay) min_pay
into qa_reporting.truv_costs_min
from truv_claims
group by 1
    '''
    cursor.execute(query)

In [8]:
for table in tables[-1:]:
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_min
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.{table}
        group by 1,2
    )
    select year, '{table}' as table_name, min(netpay), min(pay) 
    from truv_claims
    group by 1
        '''
        cursor.execute(query)

In [9]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_min;
select year, table_id_src,
        min(total_charge_amount) as min_total_charge_amount,
        min(total_allowed_amount) as min_total_allowed_amount, 
        min(total_paid_amount) as min_total_paid_amount
into qa_reporting.dw_truv_costs_min
from dw_staging.claim_header
group by 1,2
    '''
    cursor.execute(query)

In [10]:
with connection.cursor() as cursor:
    query = '''insert into qa_reporting.dw_truv_costs_min
select year, 'ALL',
        min(total_charge_amount) as min_total_charge_amount,
        min(total_allowed_amount) as min_total_allowed_amount, 
        min(total_paid_amount) as min_total_paid_amount
from dw_staging.claim_header
group by 1
    '''
    cursor.execute(query)

In [14]:
query = f'''
select a.year, b.table_name, 
        a.min_total_paid_amount as dw_min_total_paid, b.min_netpay as src_min_paid, 
        a.min_total_paid_amount-b.min_netpay as min_total_paid_difference, 
        100. * abs(a.min_total_paid_amount-b.min_netpay) / b.min_netpay as min_total_paid_difference_percentage,
        a.min_total_allowed_amount as dw_min_total_allowed, b.min_pay as src_min_allowed,
        a.min_total_allowed_amount-b.min_pay as min_total_allowed_difference, 
        100. * abs(a.min_total_allowed_amount-b.min_pay) / b.min_pay as min_total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_min a
full outer join qa_reporting.truv_costs_min b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

,year,table_name,dw_min_total_paid,src_min_paid,min_total_paid_difference,min_total_paid_difference_percentage,dw_min_total_allowed,src_min_allowed,min_total_allowed_difference,min_total_allowed_difference_percentage
14,2011,ALL,-2414961.42,-2.414961e+06,-7.741221e-06,-3.205526e-10,-8878220.00,-8.878220e+06,0.000000e+00,0.000000e+00
24,2011,ccaeo,-365085.75,-3.650858e+05,2.670288e-04,-7.314139e-08,-1598982.22,-1.598982e+06,1.862793e-03,-1.164987e-07
4,2011,ccaes,-2414961.42,-2.414961e+06,-7.741221e-06,-3.205526e-10,-8878220.00,-8.878220e+06,0.000000e+00,0.000000e+00
38,2011,mdcro,-58497.00,-5.849700e+04,0.000000e+00,0.000000e+00,-1955955.00,-1.955955e+06,0.000000e+00,0.000000e+00
25,2011,mdcrs,-1985218.00,-1.985218e+06,0.000000e+00,0.000000e+00,-1985218.00,-1.985218e+06,0.000000e+00,0.000000e+00
37,2012,ALL,-3228180.67,-3.228181e+06,-1.709000e-05,-5.294004e-10,-3228180.67,-3.228181e+06,-1.709000e-05,-5.294004e-10
20,2012,ccaeo,-431422.47,-4.314225e+05,-2.592773e-03,-6.009825e-07,-915090.00,-9.150900e+05,0.000000e+00,0.000000e+00
39,2012,ccaes,-3228180.67,-3.228181e+06,-1.709000e-05,-5.294004e-10,-3228180.67,-3.228181e+06,-1.709000e-05,-5.294004e-10
12,2012,mdcro,-107614.94,-1.076149e+05,-2.500000e-03,-2.323098e-06,-586311.96,-7.208353e+05,1.345233e+05,-1.866215e+01
36,2012,mdcrs,-859830.64,-8.598306e+05,-6.104000e-07,-7.099072e-11,-1340482.00,-1.340482e+06,0.000000e+00,0.000000e+00


### Max

In [4]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_max;
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.ccaeo
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.ccaes
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.mdcro
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.mdcrs
        group by 1,2
    )
select year, 'ALL' as table_name, 
        max(netpay) max_netpay,
        max(pay) max_pay
into qa_reporting.truv_costs_max
from truv_claims
group by 1
    '''
    cursor.execute(query)

In [5]:
for table in tables:
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_max
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.{table}
        group by 1,2
    )
    select year, '{table}' as table_name, max(netpay), max(pay) 
    from truv_claims
    group by 1
        '''
        cursor.execute(query)

In [6]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_max;
select year, table_id_src,
        max(total_charge_amount) as max_total_charge_amount,
        max(total_allowed_amount) as max_total_allowed_amount, 
        max(total_paid_amount) as max_total_paid_amount
into qa_reporting.dw_truv_costs_max
from dw_staging.claim_header
group by 1,2
    '''
    cursor.execute(query)

In [7]:
with connection.cursor() as cursor:
    query = '''insert into qa_reporting.dw_truv_costs_max
select year, 'ALL',
        max(total_charge_amount) as max_total_charge_amount,
        max(total_allowed_amount) as max_total_allowed_amount, 
        max(total_paid_amount) as max_total_paid_amount
from dw_staging.claim_header
group by 1
    '''
    cursor.execute(query)

In [10]:
query = '''
select a.year, b.table_name, 
        a.max_total_paid_amount as dw_max_total_paid, b.max_netpay as src_max_paid, 
        a.max_total_paid_amount-b.max_netpay as max_total_paid_difference, 
        100. * abs(a.max_total_paid_amount-b.max_netpay) / b.max_netpay as max_total_paid_difference_percentage,
        a.max_total_allowed_amount as dw_max_total_allowed, b.max_pay as src_max_allowed,
        a.max_total_allowed_amount-b.max_pay as max_total_allowed_difference, 
        100. * abs(a.max_total_allowed_amount-b.max_pay) / b.max_pay as max_total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_max a
full outer join qa_reporting.truv_costs_max b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,table_name,dw_max_total_paid,src_max_paid,max_total_paid_difference,max_total_paid_difference_percentage,dw_max_total_allowed,src_max_allowed,max_total_allowed_difference,max_total_allowed_difference_percentage
58,2011,ALL,3412327.00,5.917712e+07,-5.576479e+07,9.423371e+01,8878220.00,6.338144e+07,-5.450322e+07,8.599240e+01
12,2011,ccaeo,2462133.31,4.234012e+07,-3.987798e+07,9.418487e+01,2462133.31,5.000039e+07,-4.753826e+07,9.507577e+01
50,2011,ccaes,3412327.00,5.917712e+07,-5.576479e+07,9.423371e+01,8878220.00,6.338144e+07,-5.450322e+07,8.599240e+01
35,2011,mdcro,500555.00,8.280833e+05,-3.275283e+05,3.955258e+01,6106155.25,6.106155e+06,0.000000e+00,0.000000e+00
3,2011,mdcrs,1985812.00,1.985812e+06,0.000000e+00,0.000000e+00,5422836.35,5.422836e+06,2.818058e-05,5.196650e-10
28,2012,ALL,4314090.00,5.642137e+07,-5.210728e+07,9.235380e+01,4314090.00,6.030593e+07,-5.599184e+07,9.284633e+01
48,2012,ccaeo,1801357.50,3.829314e+07,-3.649179e+07,9.529587e+01,1802211.50,4.771991e+07,-4.591770e+07,9.622336e+01
13,2012,ccaes,4314090.00,5.642137e+07,-5.210728e+07,9.235380e+01,4314090.00,6.030593e+07,-5.599184e+07,9.284633e+01
7,2012,mdcro,404472.16,6.112104e+05,-2.067382e+05,3.382439e+01,1068762.00,1.068762e+06,0.000000e+00,0.000000e+00
40,2012,mdcrs,1397399.84,1.397400e+06,5.940128e-06,4.250844e-10,3174327.93,3.174328e+06,2.708564e-06,8.532717e-11


### Median

These queries take a while to run. May lose connection when executing, but the query may still be running even if connection is closed.

In [ ]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_median;
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.ccaeo
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.ccaes
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.mdcro
        group by 1,2
        union
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.mdcrs
        group by 1,2
    )
select year, 'ALL' as table_name, 
        median(netpay) median_netpay,
        median(pay) median_pay
into qa_reporting.truv_costs_median
from truv_claims
group by 1
    '''
    cursor.execute(query)

In [ ]:
for table in tables:
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_median
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.{table}
        group by 1,2
    )
    select year, '{table}' as table_name, median(netpay), median(pay) 
    from truv_claims
    group by 1
        '''
        cursor.execute(query)

In [ ]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_median;
select year, table_id_src,
        median(total_charge_amount) as median_total_charge_amount,
        median(total_allowed_amount) as median_total_allowed_amount, 
        median(total_paid_amount) as median_total_paid_amount
into qa_reporting.dw_truv_costs_median
from dw_staging.claim_header
group by 1,2
    '''
    cursor.execute(query)

In [ ]:
with connection.cursor() as cursor:
    query = '''insert into qa_reporting.dw_truv_costs_median
select year, 'ALL',
        median(total_charge_amount) as median_total_charge_amount,
        median(total_allowed_amount) as median_total_allowed_amount, 
        median(total_paid_amount) as median_total_paid_amount
from dw_staging.claim_header
group by 1
    '''
    cursor.execute(query)

In [11]:
query = f'''
select a.year, b.table_name, 
        a.median_total_paid_amount as dw_median_total_paid, b.median_netpay as src_median_paid, 
        a.median_total_paid_amount-b.median_netpay as median_total_paid_difference, 
        100. * abs(a.median_total_paid_amount-b.median_netpay) / b.median_netpay as median_total_paid_difference_percentage,
        a.median_total_allowed_amount as dw_median_total_allowed, b.median_pay as src_median_allowed,
        a.median_total_allowed_amount-b.median_pay as median_total_allowed_difference, 
        100. * abs(a.median_total_allowed_amount-b.median_pay) / b.median_pay as median_total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_median a
full outer join qa_reporting.truv_costs_median b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,table_name,dw_median_total_paid,src_median_paid,median_total_paid_difference,median_total_paid_difference_percentage,dw_median_total_allowed,src_median_allowed,median_total_allowed_difference,median_total_allowed_difference_percentage
51,2011,ALL,55.72,55.709991,1.000854e-02,1.796544e-02,95.00,95.000000,0.000000e+00,0.000000e+00
54,2011,ccaeo,64.99,64.979980,1.001953e-02,1.541941e-02,93.13,93.109985,2.001465e-02,2.149571e-02
53,2011,ccaes,141.80,141.790000,1.000000e-02,7.052684e-03,172.84,172.840000,1.490150e-10,8.621559e-11
52,2011,mdcro,18.00,18.000000,0.000000e+00,0.000000e+00,95.78,95.750000,3.000000e-02,3.133159e-02
50,2011,mdcrs,24.26,24.250000,1.000000e-02,4.123711e-02,106.70,106.700000,7.450609e-10,6.982764e-10
1,2012,ALL,55.71,55.709976,2.380371e-05,4.272791e-05,97.00,97.000000,0.000000e+00,0.000000e+00
4,2012,ccaeo,64.95,64.939995,1.000519e-02,1.540682e-02,95.96,95.949951,1.004883e-02,1.047299e-02
3,2012,ccaes,141.30,141.300000,1.117598e-09,7.909402e-10,176.22,176.200000,2.000000e-02,1.135074e-02
0,2012,mdcro,16.20,16.199997,3.051758e-06,1.883801e-05,96.01,96.000000,1.000000e-02,1.041667e-02
2,2012,mdcrs,21.02,21.020000,2.142038e-10,1.019047e-09,103.15,103.150000,5.587992e-10,5.417346e-10


## 